In [4]:
from PIL import Image, ImageFilter, ImageOps
import numpy as cp
import matplotlib.pyplot as plt
import cv2

In [5]:
import cupy as cp
from PIL import Image
import cv2
from scipy import stats

In [6]:
def analyze_dni_image(image):
    """
    Analiza múltiples características de la imagen para detectar si es un DNI original
    o una impresión/fotocopia.
    
    :param image: Imagen PIL
    :return: Dict con métricas y análisis
    """
    # Convertir a escala de grises si no lo está
    gray_image = image.convert('L')
    img_array = cp.array(gray_image)
    
    # Convertir a CV2 para algunos análisis
    cv2_image = cp.array(gray_image)
    
    # 1. Análisis de patrones de impresión
    def detect_printer_patterns():
        # Detectar patrones regulares típicos de impresoras
        # Usar FFT para detectar patrones periódicos
        f = cp.fft.fft2(img_array)
        fshift = cp.fft.fftshift(f)
        magnitude_spectrum = 20 * cp.log(cp.abs(fshift))
        # Buscar picos regulares en el espectro de frecuencia
        periodic_score = cp.std(magnitude_spectrum)
        return periodic_score
    
    # 2. Análisis de bordes
    def analyze_edges():
        edges = cv2.Canny(cv2_image, 100, 200)
        edge_density = cp.sum(edges > 0) / edges.size
        return edge_density
    
    # 3. Análisis de textura
    def analyze_texture():
        # GLCM (Gray-Level Co-Occurrence Matrix)
        glcm = cp.zeros((256, 256))
        rows, cols = img_array.shape
        for i in range(rows-1):
            for j in range(cols-1):
                glcm[img_array[i,j], img_array[i,j+1]] += 1
        
        # Normalizar GLCM
        glcm = glcm / cp.sum(glcm)
        
        # Calcular características de textura
        contrast = cp.sum(cp.square(cp.arange(256)[:,None] - cp.arange(256)[None,:]) * glcm)
        homogeneity = cp.sum(glcm / (1 + cp.square(cp.arange(256)[:,None] - cp.arange(256)[None,:])))
        
        return contrast, homogeneity
    
    # 4. Análisis de ruido
    def analyze_noise():
        # Calcular la desviación estándar local
        local_std = cp.std(img_array)
        
        # Aplicar filtro de mediana y comparar con original
        median_filtered = cv2.medianBlur(cv2_image, 3)
        noise_diff = cp.mean(cp.abs(cv2_image - median_filtered))
        
        return local_std, noise_diff
    
    # 5. Análisis de niveles de gris
    def analyze_gray_levels():
        histogram = cp.histogram(img_array, bins=256, range=(0,256))[0]
        histogram = histogram / cp.sum(histogram)
        
        # Calcular entropía
        entropy = -cp.sum(histogram * cp.log2(histogram + 1e-10))
        
        # Calcular momentos estadísticos
        mean = cp.mean(img_array)
        std = cp.std(img_array)
        skewness = stats.skew(img_array.ravel())
        kurtosis = stats.kurtosis(img_array.ravel())
        
        return entropy, mean, std, skewness, kurtosis
    
        # Ejecutar todos los análisis
    metrics = {
        'printer_pattern_score': detect_printer_patterns(),
        'edge_density': analyze_edges(),
        'texture_contrast': analyze_texture()[0],
        'texture_homogeneity': analyze_texture()[1],
        'noise_level': analyze_noise()[0],
        'noise_difference': analyze_noise()[1],
        'gray_entropy': analyze_gray_levels()[0],
        'gray_mean': analyze_gray_levels()[1],
        'gray_std': analyze_gray_levels()[2],
        'gray_skewness': analyze_gray_levels()[3],
        'gray_kurtosis': analyze_gray_levels()[4]
    }
    
    return metrics

def classify_dni(image, thresholds=None):
    """
    Clasifica un DNI como original o impresión basándose en múltiples características.
    
    :param image: Imagen PIL del DNI
    :param thresholds: Dict con umbrales para cada métrica (opcional)
    :return: 'valida' o 'invalida' y métricas
    """
    # Analizar la imagen
    metrics = analyze_dni_image(image)
    
    # Umbrales por defecto (estos deberían ajustarse con un conjunto de entrenamiento)
    default_thresholds = {
        'printer_pattern_score': 50,
        'edge_density': 0.1,
        'texture_contrast': 100,
        'noise_level': 20,
        'gray_entropy': 7
    }
    
    thresholds = thresholds or default_thresholds
    
    # Sistema de puntuación
    score = 0
    
    # Reglas de clasificación invertidas
    if metrics['printer_pattern_score'] <= thresholds['printer_pattern_score']:
        score -= 1  # Patrones menos regulares sugieren un DNI original
    
    if metrics['edge_density'] >= thresholds['edge_density']:
        score -= 1  # Bordes más definidos sugieren un DNI original
    
    if metrics['texture_contrast'] >= thresholds['texture_contrast']:
        score -= 1  # Mayor contraste de textura sugiere un DNI original
    
    if metrics['noise_level'] >= thresholds['noise_level']:
        score -= 1  # Mayor nivel de ruido sugiere un DNI original
    
    if metrics['gray_entropy'] >= thresholds['gray_entropy']:
        score -= 1  # Mayor entropía sugiere un DNI original
    
    # Clasificación final invertida
    result = "valida" if score <= -3 else "invalida"
    
    return result, metrics

In [7]:
# image_path_frente = 'prueba/imagen_1_frente.jpeg'
# image_path_dorso = 'prueba/imagen_1_dorso.jpeg'
# 
# image_frente = Image.open(image_path_frente)
# image_dorso = Image.open(image_path_dorso)
# 
# # Convertir imágenes a blanco y negro y binarizadas
# image_frente_blanco_y_negro = image_frente.convert("L")
# image_dorso_blanco_y_negro = image_dorso.convert("L")
# 
# # Guardar las imágenes blanco y negro
# image_frente_blanco_y_negro.save('prueba/validas/imagen_1_blanco_y_negro_frente.jpeg')
# image_dorso_blanco_y_negro.save('prueba/validas/imagen_1_blanco_y_negro_dorso.jpeg')
# 
# # Aplicar la función de ruido y binarización a ambas imágenes
# image_frente_fotocopia = make_background_white_and_add_noise_and_binarize(image_frente)
# image_dorso_fotocopia = make_background_white_and_add_noise_and_binarize(image_dorso)
# 
# # Guardar las imágenes procesadas (binarizadas con ruido)
# image_frente_fotocopia.save('prueba/invalidas/imagen_1_frente_fotocopia.jpeg')
# image_dorso_fotocopia.save('prueba/invalidas/imagen_1_dorso_fotocopia.jpeg')
# 
# # Crear una figura con 2 filas y 2 columnas para las imágenes
# fig, axs = plt.subplots(2, 2, figsize=(12, 12))
# 
# # Mostrar el "frente" en blanco y negro y binarizado
# axs[0, 0].imshow(image_frente_blanco_y_negro, cmap='gray')
# axs[0, 0].set_title("Frente - Blanco y Negro")
# axs[0, 0].axis("off")
# 
# axs[0, 1].imshow(image_frente_fotocopia, cmap='gray')
# axs[0, 1].set_title("Frente - Binarizado con Ruido")
# axs[0, 1].axis("off")
# 
# # Mostrar el "dorso" en blanco y negro y binarizado
# axs[1, 0].imshow(image_dorso_blanco_y_negro, cmap='gray')
# axs[1, 0].set_title("Dorso - Blanco y Negro")
# axs[1, 0].axis("off")
# 
# axs[1, 1].imshow(image_dorso_fotocopia, cmap='gray')
# axs[1, 1].set_title("Dorso - Binarizado con Ruido")
# axs[1, 1].axis("off")
# 
# # Ajustar la visualización y mostrar el gráfico
# plt.tight_layout()
# plt.show()

In [8]:
# print(f"Imagen valida frente: {classify_image(image_frente)}")
# print(f"Imagen valida dorso: {classify_image(image_dorso)}")
# print(f"Imagen invalida frente: {classify_image(image_frente_fotocopia)}")
# print(f"Imagen invalida dorso: {classify_image(image_dorso_fotocopia)}")

In [9]:
import numpy as cp
from PIL import Image
from pathlib import Path

def evaluate_classifier_precision(valid_images_path, invalid_images_path, binarization_threshold=50):
    """
    Evalúa la precisión del clasificador de imágenes procesando dos conjuntos de imágenes:
    uno de imágenes válidas y otro de imágenes inválidas.
    
    :param valid_images_path: Ruta a la carpeta con imágenes válidas
    :param invalid_images_path: Ruta a la carpeta con imágenes inválidas
    :param binarization_threshold: Umbral para la clasificación
    :return: Dict con métricas de precisión
    """
    # Contadores para los resultados
    true_positives = 0  # Imágenes válidas correctamente clasificadas como válidas
    true_negatives = 0  # Imágenes inválidas correctamente clasificadas como inválidas
    false_positives = 0  # Imágenes inválidas clasificadas incorrectamente como válidas
    false_negatives = 0  # Imágenes válidas clasificadas incorrectamente como inválidas
    
    # Procesar imágenes válidas
    valid_path = Path(valid_images_path)
    for img_path in valid_path.glob('*'):
        try:
            image = Image.open(img_path)
            result = classify_dni(image, binarization_threshold)
            if result == "valida":
                true_positives += 1
            else:
                print(img_path)
                false_negatives += 1
                # print(img_path)
        except Exception as e:
            print(f"Error procesando imagen valida {img_path}: {str(e)}")
    
    # Procesar imágenes inválidas
    invalid_path = Path(invalid_images_path)
    for img_path in invalid_path.glob('*'):
        try:
            image = Image.open(img_path)
            result = classify_dni(image, binarization_threshold)
            if result == "invalida":
                true_negatives += 1
            else:
                false_positives += 1
                # print(img_path)
        except Exception as e:
            print(f"Error procesando imagen invalida {img_path}: {str(e)}")
    
    # Calcular métricas
    total_valid = true_positives + false_negatives
    total_invalid = true_negatives + false_positives
    
    metrics = {
        "true_positives": true_positives,
        "true_negatives": true_negatives,
        "false_positives": false_positives,
        "false_negatives": false_negatives,
        "precision_valid": (true_positives / total_valid) * 100 if total_valid > 0 else 0,
        "precision_invalid": (true_negatives / total_invalid) * 100 if total_invalid > 0 else 0,
        "total_precision": ((true_positives + true_negatives) / (total_valid + total_invalid)) * 100
    }
    
    return metrics

def print_results(metrics):
    """
    Imprime los resultados de la evaluación de manera formateada.
    
    :param metrics: Dict con las métricas calculadas
    """
    print("\nResultados de la evaluación:")
    print("-" * 50)
    print(f"Imágenes válidas correctamente clasificadas: {metrics['true_positives']}/90")
    print(f"Imágenes inválidas correctamente clasificadas: {metrics['true_negatives']}/90")
    print(f"\cprecisión para imágenes válidas: {metrics['precision_valid']:.2f}%")
    print(f"Precisión para imágenes inválidas: {metrics['precision_invalid']:.2f}%")
    print(f"Precisión total: {metrics['total_precision']:.2f}%")
    print("\nMatriz de confusión:")
    print("-" * 50)
    print(f"                  Predicho válida    Predicho inválida")
    print(f"Real válida      {metrics['true_positives']:^16d} {metrics['false_negatives']:^18d}")
    print(f"Real inválida    {metrics['false_positives']:^16d} {metrics['true_negatives']:^18d}")

<>:76: SyntaxWarning: invalid escape sequence '\c'
<>:76: SyntaxWarning: invalid escape sequence '\c'
/tmp/ipykernel_6475/1577776098.py:76: SyntaxWarning: invalid escape sequence '\c'
  print(f"\cprecisión para imágenes válidas: {metrics['precision_valid']:.2f}%")


In [10]:
print_results(evaluate_classifier_precision('DNI-Validos', 'DNI-Invalidos', binarization_threshold=None))

DNI-Validos/Documento_Nacional_Identidad_Dorso_Versión_1_9ecb9ca5-7ca5-46a5-80df-9ec59505d2b5.jpeg
DNI-Validos/Documento_Nacional_Identidad_Frente_Versión_1_df120cb0-e849-4af1-9276-2f834840593f.jpg
DNI-Validos/Documento_Nacional_Identidad_Dorso_Versión_1_1a9ddb03-b8f1-47dd-aae4-b1d7f2627b3c.jpeg
DNI-Validos/Documento_Nacional_Identidad_Dorso_Versión_1_558dd269-0f40-4ffa-b142-daba87c4a7d1.jpeg
DNI-Validos/Documento_Nacional_Identidad_Frente_Versión_1_5aa8c9e9-1508-49e8-b507-a9d7bbaf602e.jpeg
DNI-Validos/Documento_Nacional_Identidad_Dorso_Versión_1_79e60cac-4742-45b8-bd87-5781caf86755.jpeg
DNI-Validos/Documento_Nacional_Identidad_Frente_Versión_1_893e0f46-f89e-4e03-afe4-6d09c3a55da3.jpeg
DNI-Validos/Documento_Nacional_Identidad_Dorso_Versión_1_c344d13b-a8cf-4f40-98c4-2eccab882a02.jpeg
DNI-Validos/Documento_Nacional_Identidad_Dorso_Versión_1_8858fc50-4bb6-4fdd-a382-4e2cbf6c3c93.jpeg
DNI-Validos/Documento_Nacional_Identidad_Dorso_Versión_1_ff104644-3257-4ddc-84f5-55e444de7946.jpeg
DNI-Vali